In [190]:
import pandas as pd
import os

In [191]:
demand_df = pd.read_csv(os.path.join(os.getcwd(), "CAISO_demandtrends_modified.csv"))
net_demand_df = pd.read_csv(os.path.join(os.getcwd(), "CAISO_netdemandtrends_modified.csv"))
emissions_df =  pd.read_csv(os.path.join(os.getcwd(), "CAISO_Data_2019_2021_NN.csv"), index_col=0)

C:\Users\nicklein\AppData\Local\Temp\ipykernel_3928\864816642.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  net_demand_df = pd.read_csv(os.path.join(os.getcwd(), "CAISO_netdemandtrends_modified.csv"))


In [192]:
demand_df['time'] =  pd.to_datetime(demand_df["time"], format="%m/%d/%y %H:%M", errors='coerce')
demand_df = demand_df.dropna(subset=['time'])
demand_df = demand_df.set_index("time")

In [193]:
net_demand_df['time'] =  pd.to_datetime(net_demand_df["time"], format="%m/%d/%y %H:%M", errors='coerce')
net_demand_df = net_demand_df.dropna(subset=['time'])
net_demand_df = net_demand_df.set_index("time")

In [194]:
emissions_df.index = pd.to_datetime(emissions_df.index)
earliest_datetime, latest_datetime = min(emissions_df.index), max(emissions_df.index)

In [195]:
# jk we want to include the first hour of 2022
latest_datetime = pd.to_datetime('1/1/22 0:55', format="%m/%d/%y %H:%M")

In [196]:
demand_df = demand_df.loc[(demand_df.index >= earliest_datetime) & (demand_df.index <= latest_datetime)]

In [197]:
net_demand_df = net_demand_df.loc[(net_demand_df.index >= earliest_datetime) & (net_demand_df.index <= latest_datetime)]

net demand table has some non-float values. Fix that first, filling in non-floats with nan

In [198]:
net_demand_df.loc[:,"Day-ahead net forecast"] = pd.to_numeric(net_demand_df.loc[:,"Day-ahead net forecast"], errors="coerce")

C:\Users\nicklein\AppData\Local\Temp\ipykernel_3928\101297538.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  net_demand_df.loc[:,"Day-ahead net forecast"] = pd.to_numeric(net_demand_df.loc[:,"Day-ahead net forecast"], errors="coerce")


get average values for each hour

**Note** There are nan values in these tables. The behavior of resample here is the following: If there are any non-nan values in the hour, those are used to compute the mean for the hour. If there are no non-nan values in the hour, the mean for that hour is recorded as nan.

**I also note...** that on some of the hours around DST shift where day-ahead projection is all nan, the hour ahead projection is all 0.0. This will be very bad for prediction. In this case, it might be a good idea to just also drop those, treating them like there is no available prediction at that point. I haven thouroughly looked at the data though so there may be more things like this to try to catch (e.g. by searching for values in ranges that could be easily filtered out as bad projections.

## Stepp todo here - filter out projections where we have a non-nan projected value but the value is trivially a bad projection

Do this by turning those trivially bad values to 0.  
I have already done a little of this. If you want to do more, do it here.  
Might also make sense to do this after the resampling as then you'll have fewer values to look at.

In [199]:
col_dict = {"day-ahead demand": demand_df.loc[:,'Day-ahead forecast'],
            "hour-ahead demand": demand_df.loc[:,'Hour-ahead forecast Demand'],
            "day-ahead net demand": net_demand_df.loc[:,'Day-ahead net forecast'],
            "hour-ahead net demand": net_demand_df.loc[:,'Hour-ahead forecast Net Demand']
           }

In [226]:
net_demand_df.loc[net_demand_df.loc[:,'Day-ahead net forecast'] < 0]

,Day-ahead net forecast,Hour-ahead forecast Net Demand,Demand
time,,,
2019-10-03 14:40:00,-4158.0,22254.0,25076.0
2019-10-03 14:50:00,-13200.0,22601.0,25153.0
2019-10-03 14:55:00,-14006.0,22601.0,25221.0
2019-10-03 15:00:00,-5648.0,22601.0,25303.0
2021-04-04 16:25:00,-3844.0,22373.0,18873.0
2021-07-07 11:55:00,-38682.0,35406.0,29021.0


In [225]:
lb = 1000
print(f"Number of points projected valued <= {lb}")
for name, vals in col_dict.items():
    print(f"\t{name}: {sum(vals <= lb)}")

Number of points projected valued <= 1000
	day-ahead demand: 0
	hour-ahead demand: 12
	day-ahead net demand: 6
	hour-ahead net demand: 0


In [ ]:
# Continue with further filtering here...

Continuing with the resampling

In [177]:
demand_df = demand_df.resample('h').mean()
net_demand_df = net_demand_df.resample('h').mean()

In [178]:
demand_df.head()

,Day-ahead forecast,Hour-ahead forecast Demand,Demand
time,,,
2019-01-01 00:00:00,22452.500000,21822.166667,21851.416667
2019-01-01 01:00:00,21520.583333,21066.083333,21099.666667
2019-01-01 02:00:00,20856.416667,20457.000000,20495.416667
2019-01-01 03:00:00,20448.250000,20128.666667,20147.500000
2019-01-01 04:00:00,20368.333333,20133.750000,20144.416667


In [179]:
net_demand_df.head()

,Day-ahead net forecast,Hour-ahead forecast Net Demand,Demand
time,,,
2019-01-01 00:00:00,18389.727273,22389.750000,21851.416667
2019-01-01 01:00:00,17704.583333,21748.833333,21099.666667
2019-01-01 02:00:00,16988.833333,21245.000000,20495.416667
2019-01-01 03:00:00,16406.750000,21218.083333,20147.500000
2019-01-01 04:00:00,16436.750000,21837.666667,20144.416667


Checking how many values we have nan for...

In [180]:
print("Number of hours we are missing projected values fore...")
print(f"\tday-ahead demand: {sum(pd.isna(demand_df.loc[:,'Day-ahead forecast']))}")
print(f"\thour-ahead demand: {sum(pd.isna(demand_df.loc[:,'Hour-ahead forecast Demand']))}")
print(f"\tday-ahead net demand: {sum(pd.isna(net_demand_df.loc[:,'Day-ahead net forecast']))}")
print(f"\thour-ahead net demand: {sum(pd.isna(net_demand_df.loc[:,'Hour-ahead forecast Net Demand']))}")

Number of hours we are missing projected values fore...
	day-ahead demand: 3
	hour-ahead demand: 2
	day-ahead net demand: 288
	hour-ahead net demand: 27


Check that dropping rows and aggregation was done correctly by comparing the indexes of our different tables

In [181]:
len(demand_df) == len(net_demand_df) == len(emissions_df)

True

In [182]:
all(demand_df.index == net_demand_df.index) and all(demand_df.index == emissions_df.index)

True

Move projected values over to one table with the rest of the data

In [183]:
emissions_df.loc[:,"Day_Ahead_Forecast_Demand"] = demand_df.loc[:,"Day-ahead forecast"]
emissions_df.loc[:,"Hour_Ahead_Forecast_Demand"] = demand_df.loc[:,"Hour-ahead forecast Demand"]

In [184]:
emissions_df.loc[:,"Day_Ahead_Forecast_VRE"] = demand_df.loc[:,"Day-ahead forecast"] \
                                                - net_demand_df.loc[:,"Day-ahead net forecast"]
emissions_df.loc[:,"Hour_Ahead_Forecast_VRE"] = demand_df.loc[:,"Hour-ahead forecast Demand"] \
                                                - net_demand_df.loc[:,"Hour-ahead forecast Net Demand"]

Verification that subtraction of two terms where either is nan gives you a nan in pandas:

In [185]:
import numpy as np

In [186]:
pd.Series([np.nan, 1.2, np.nan, 2.3]) - pd.Series([np.nan, 1.2, 1000, np.nan])

0    NaN
1    0.0
2    NaN
3    NaN
dtype: float64

Resulting number of missing values in VRE...

In [187]:
print("Number of hours we are missing projected values for VRE (either demand or net-demand is nan for that hour)...")
print(f"\tday-ahead VRE: {sum(pd.isna(emissions_df.loc[:,'Day_Ahead_Forecast_VRE']))}")
print(f"\thour-ahead VRE: {sum(pd.isna(emissions_df.loc[:,'Hour_Ahead_Forecast_VRE']))}")

Number of hours we are missing projected values for VRE (either demand or net-demand is nan for that hour)...
	day-ahead VRE: 291
	hour-ahead VRE: 29


Okay, so each hour that we had all nans for in the demand projections were also hours that we had all nans for in the net-demand projections. makes sense.

In [188]:
emissions_df

,Load,Net Load,Total_CO2_Emissions,Total_SO2_Emissions,Total_NOX_Emissions,VRE,delta_Load,delta_Net_Load,delta_Total_CO2_Emissions,delta_Total_SO2_Emissions,delta_Total_NOX_Emissions,delta_VRE,Day_Ahead_Forecast_Demand,Hour_Ahead_Forecast_Demand,Day_Ahead_Forecast_VRE,Hour_Ahead_Forecast_VRE
2019-01-01 00:00:00,22822.964472,20502.358502,5.103942e+06,425.327933,1632.821698,2320.593616,-1285.054865,-1255.110267,-337029.794143,-24.142180,-98.530417,-29.956338,22452.500000,21822.166667,4062.772727,-567.583333
2019-01-01 01:00:00,21879.620618,19606.836908,4.867578e+06,404.315852,1557.650531,2272.780097,-944.689268,-896.922625,-243021.833700,-21.594332,-76.882199,-47.757034,21520.583333,21066.083333,3816.000000,-682.750000
2019-01-01 02:00:00,21257.454020,19056.267637,4.723101e+06,383.695714,1496.197481,2201.182455,-614.641020,-545.206677,-144846.797503,-20.952957,-61.425638,-69.432802,20856.416667,20457.000000,3867.583333,-788.000000
2019-01-01 03:00:00,20974.800758,18871.418601,4.693112e+06,380.561848,1466.329836,2103.388502,-281.391674,-191.565227,-24776.569759,-2.164379,-28.118595,-89.815150,20448.250000,20128.666667,4041.500000,-1089.416667
2019-01-01 04:00:00,20327.083333,18012.666667,5.032423e+06,711.911968,2391.657870,2314.666667,30.416667,74.416667,49254.136541,69.703951,71.970840,-43.916667,20368.333333,20133.750000,3931.583333,-1703.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 20:00:00,25768.833333,23072.083333,5.669181e+06,379.791774,1258.958412,2696.666667,-746.916667,-704.333333,56445.099855,9.455888,15.439436,-42.500000,26192.833333,25565.750000,438.833333,-775.750000
2021-12-31 21:00:00,25023.083333,22046.666667,5.470524e+06,360.779828,1250.710850,2976.333333,-745.750000,-1025.416667,-198657.681862,-19.011945,-8.247563,279.666667,25507.416667,24894.916667,936.416667,-672.833333
2021-12-31 22:00:00,24080.333333,20955.166667,5.290431e+06,402.956860,1279.316169,3125.250000,-942.750000,-1091.500000,-180092.764556,42.177031,28.605320,148.916667,24531.583333,23950.666667,1439.583333,-236.583333
2021-12-31 23:00:00,24110.687979,21757.807344,5.442142e+06,450.015950,1732.354803,2352.880251,-1840.714198,-1817.217265,-382819.120048,-19.647903,-94.140767,-23.501687,23562.500000,23007.333333,1822.500000,180.583333


In [189]:
emissions_df.to_csv(os.path.join(os.getcwd(), "CAISO_Data_2019_2021_NN.with_historical_projections.csv"))